[x] Utilize o dataset Titanic

[x] - Realize o pre-processamento dos dados: limpeza, tratamento de valores faltantes, conversão de dados categóricos vetorização, normalização.

[] - Avalie um rede MLP utilizando o método de validação-cruzada com holdout. Varie o número de camadas ocultas (1,2,3) e de neurônios em cada camada oculta (16, 32,64). Use 10% do conjunto de treino para validação.

[] - Use como função de ativação nas camadas ocultas a função ReLU e na camada de saída a softmax.

[] - Para o treinamento utilize RMSprop como otimizador, entropia cruzada como função de loss (sparse_categorical_crossentropy). 

[] - Treine utilizando parada antecidapa: EarlyStopping como callback. Veja exemplo. 

[] - Monitore a acurácia durante o treino. Armazene as informações de treino usando history = model.fit(...). Imprima um gráfico que apresente a variação da acurácia de treino e de validação no decorrer das épocas. Imprima também um outro gráfico para a loss. Você encontra exemplos de como plotar essa informção aqui.

[] - Avalie o melhor modelo com os dados de teste: model.evaluate(...)

In [ ]:
import pandas as pd
train_data = pd.read_csv("./titanic/train.csv")
test_data = pd.read_csv("./titanic/test.csv")

train_data.head(5)

In [ ]:
train_data.info()

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

### Muitos valores da idade estão faltando, logo, tenho duas opções:
    [] Remover a coluna.
    [x] Colocar algum valor substituto, já que a idade na hora de salvar foi importante (Ex: crianças foram salvas primeiro).

In [ ]:
def adicionar_idade(row):
    if pd.isnull(row["Age"]):
        return newAge
    else:
        return row["Age"]

newAge = train_data["Age"].median() # Pego a mediana das idades.
train_data['Age'] = train_data.apply(adicionar_idade, axis = 1)

newAge = test_data["Age"].median() # Pego a mediana das idades (Não a média).
test_data['Age'] = test_data.apply(adicionar_idade, axis = 1)

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

### Será que a cabine que a pessoa estava tem relação com sua sobrevivência?

In [ ]:
cabin_survival = train_data.groupby("Cabin")["Survived"].mean()
print(cabin_survival)
# Percebemos que dependendo da Cabine a pessoa tem mais chances de sobreviver, algumas 100%, outras 0%...

In [ ]:
cabinNames = set(train_data['Cabin'])
mapping = [i for i in range(len(cabinNames))]
train_data['Cabin'].replace(cabinNames, mapping, inplace=True) # Mapeando para inteiro

cabinNames = set(test_data['Cabin'])
mapping = [i for i in range(len(cabinNames))]
test_data['Cabin'].replace(cabinNames, mapping, inplace=True) # Mapeando para inteiro

train_data['Cabin']

In [ ]:
train_data.isnull().sum()

In [ ]:
test_data.isnull().sum()

### Lidando com o último atributo faltante (Fare)

In [ ]:
fare_median = test_data["Fare"].median()
test_data["Fare"].fillna(fare_median, inplace=True) # Preenchendo valores nulos com a mediana

test_data.isnull().sum()

### Será que o porto de embarque tem relação com sua sobrevivência?

In [ ]:
embarked_survival = train_data.groupby("Embarked")["Survived"].mean()
print(embarked_survival)

In [ ]:
embarkedNames = set(train_data['Embarked'])
mapping = [i for i in range(len(embarkedNames))]
train_data['Embarked'].replace(embarkedNames, mapping, inplace=True) # Mapeando para inteiro

embarkedNames = set(test_data['Embarked'])
mapping = [i for i in range(len(embarkedNames))]
test_data['Embarked'].replace(embarkedNames, mapping, inplace=True) # Mapeando para inteiro

train_data.isnull().sum()

In [ ]:
train_data.head(10)

### Convertendo atributo categórico Sex para numérico

In [ ]:
train_data["Sex"].replace(["male", "female"], [0, 1], inplace=True)
test_data["Sex"].replace(["male", "female"], [0, 1], inplace=True)

train_data.head(10)

### Importando os dados

In [ ]:
# Apagando colunas inúteis para a análise (Colunas como nome e Ticket, além de Survived já que é usada no y_train)
X_train = train_data.drop(['Survived', 'Name', 'Ticket','PassengerId'], axis=1)
# Pegando o y
y_train = train_data['Survived']

# Apagando colunas inúteis para a análise nos dados de teste
X_test = test_data.drop(['Name', 'Ticket','PassengerId'], axis=1)

X_test.info()

### Normalizando dados de treino, validação e teste

In [ ]:
from sklearn.preprocessing import StandardScaler

normalized = StandardScaler()
normalized.fit(X_train)

X_train_std = normalized.transform(X_train)
X_test_std = normalized.transform(X_test)

### Treinando o modelo

In [ ]:
from tensorflow import keras
from keras.layers import Dense
from keras.models import Sequential
import numpy as np
import matplotlib.pyplot as plt

callback = keras.callbacks.EarlyStopping(monitor="val_loss", patience=20)

maxAcc = float("-inf")
numLayers = [1, 2, 3]
numNeurons = [16, 32, 64]

bestNumLayers = numLayers[0]
bestNumNeuron = numNeurons[0]


for nLayer in numLayers:
    for nNeuron in numNeurons:
        inter = 0
        sqLayers = [Dense(nNeuron, activation="relu")] * (nLayer - 1) + [
            Dense(2, activation="softmax")
        ]
        model = Sequential(sqLayers)
        model.compile(
            optimizer="rmsprop",
            loss="sparse_categorical_crossentropy",
            metrics=["accuracy", "top_k_categorical_accuracy"],
        )
        history = model.fit(
            x=X_train_std,
            y=y_train,
            epochs=100,
            callbacks=[callback],
            validation_split=0.1,
            verbose=0,
        )

        if np.mean(history.history["val_accuracy"]) > maxAcc:
            bestNLayers = nLayer
            bestNNeuron = nNeuron

        print(f"Quantidade de camadas ocultas: {nLayer}")
        print(f"Quantidade de neurônios nesta camada: {nNeuron}\n\n")

        #  Variação da acurácia de treino e de validação
        plt.plot(history.history["accuracy"])
        plt.plot(history.history["val_accuracy"])
        plt.title("Acurácia do modelo")
        plt.ylabel("Acurácia")
        plt.xlabel("Época")
        plt.legend(["Treino", "Validação"], loc="bottom right")
        plt.show()

        # Variação do loss
        plt.plot(history.history["loss"])
        plt.plot(history.history["val_loss"])
        plt.title("Perdas do modelo")
        plt.ylabel("Perca")
        plt.xlabel("Época")
        plt.legend(["Treino", "Validação"], loc="bottom right")
        plt.show()

### Realizando a predição

In [ ]:
# Apagando colunas inúteis para a análise (Colunas como nome e Ticket, além de Survived já que é usada no y_test)
# y_pred = dt.predict(X_test)
# to_submit = pd.concat([test_data.PassengerId, pd.DataFrame({"Survived": y_pred})], axis=1)

# to_submit

### Exportando para fazer a predição no site

In [ ]:
# to_submit.to_csv("output.csv", index = False)